In [107]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
def draw_lines_solid(img, lines, color=[255, 0, 0], thickness=5):
    #average the m (slope) and b of the lines detected by Hough transform
    m_left = []
    m_right = []
    b_right = []
    b_left = []
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            m = (y1-y2)/(x1-x2)
            b = y1 - x1*m
            #seperate the left and right lines
            if m > 0:
                m_right.append(m)
                b_right.append(b)
            else:
                m_left.append(m)
                b_left.append(b)
    
    #deal with the no-line-detected case
    if len(m_right):
        m_right_average = np.mean(m_right)
        b_right_average = np.mean(b_right)
        y1_right = 540
        x1_right = int((y1_right - b_right_average)/m_right_average)
        y2_right = 320
        x2_right = int((y2_right - b_right_average)/m_right_average)
    else:
        y1_left = 0
        x1_left = 0 
        y2_left = 0
        x2_left = 0

    if len(m_left):
        m_left_average = np.mean(m_left)
        b_left_average = np.mean(b_left)
        y1_left = 540
        x1_left = int((y1_left - b_left_average)/m_left_average)
        y2_left = 320
        x2_left = int((y2_left - b_left_average)/m_left_average)
    else:
        y1_left = 0
        x1_left = 0 
        y2_left = 0
        x2_left = 0       
  
    lines_extrapolated = np.array([[x1_left,y1_left,x2_left,y2_left],[x1_right,y1_right,x2_right,y2_right]])
    
    #draw the left and right extropolated lines
    line_image_extrapolated = cv2.line(img, (x1_left, y1_left), (x2_left, y2_left), color, thickness)
    line_image_extrapolated = cv2.line(img, (x1_right, y1_right), (x2_right, y2_right), color, thickness)
    #plt.imshow(line_image_extrapolated)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    #draw the raw lines
    #draw_lines(line_img, lines)
    
    #draw the extrapolate lines
    draw_lines_solid(line_img, lines)
    
    return line_img

def process_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    edges_image = cv2.Canny(image, 100, 200)
    imshape = image.shape
    vertices = np.array([[(100,imshape[0]),(450, 330), (550, 330), (850,imshape[0])]], dtype=np.int32)
    masked_image = region_of_interest(edges_image, vertices)
    
    #set the parameters for Hough transform
    rho = 2
    theta = np.pi/180
    threshold = 25
    min_line_len = 40
    max_line_gap = 20
    
    line_image = hough_lines(masked_image, rho, theta, threshold, min_line_len, max_line_gap)
    combo_image = cv2.addWeighted(image, 0.8, line_image, 1, 0) 
    
    return combo_image
    
#image = mpimg.imread('test_images/solidWhiteRight.jpg')
#result_image = process_image(image)
#plt.imshow(result_image)


In [108]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))


[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


 99%|█████████▉| 125/126 [00:03<00:00, 37.93it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 2.62 s, sys: 457 ms, total: 3.08 s
Wall time: 4.09 s
